In [1]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv
load_dotenv()

from rs import Message, Config, chat
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))

def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"
    
    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)

            custom_action_button = widgets.Button(description="CUSTOM")
            custom_action_button.on_click(on_custom_action)
            buttons.append(custom_action_button)

    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/config_cn.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))


你被一位富商雇佣，调查阿卡姆郊外的一座闹鬼的豪宅。
传说这座豪宅被诅咒，任何进入的人都不会再回来。
你已经得到了豪宅的地图和一些基本装备，以帮助你的调查。
你的目标是找出诅咒背后的真相，并结束它。

你站在闹鬼的豪宅前。
这座豪宅古老而破旧，墙上爬满了爬山虎，窗户被钉上了木板。
前门微微敞开，邀请你进入。

你要做什么？

possible actions:
1. 进入豪宅。
2. 拍摄豪宅外部的照片。
3. 与管家交谈。(可能需要一些说服力，以便让你获取更多信息)


I select option 1. TURN: 1 


你鼓起勇气，迈步走进了豪宅的大厅。
门在你身后轻轻关上，豪宅内部的灰尘和阴暗让你不由得打了一个寒颤。
大厅内部装饰豪华，但多年的废弃让它显得有些破败。
你注意到有三个方向可以去探索：左边是一间餐厅，右边是楼梯通往上楼，正前方则是一扇通往豪宅后部的门。

你要去哪里？

possible actions:
1. 去餐厅。
2. 上楼。
3. 去豪宅后部。

I select option 3. TURN: 2 


你选择前往豪宅的后部。
穿过一条长长的走廊，两边挂着一些模糊的肖像画，似乎是这座豪宅过去的主人。
走廊的尽头有一扇门，你推开它，发现自己来到了一个宽敞的图书室。
图书室里到处都是满满的书架，尘封的书籍散发出一股旧时代的气息。
你注意到书架之间有一张桌子，上面放着一本敞开的日记和一盏昏暗的台灯。

你要做什么？

possible actions:
1. 仔细阅读日记。
2. 检查书架上是否有隐藏的通道或机关。
3. 离开图书室，探索其他地方。

I select option 1. TURN: 3 


你走向桌子，坐下来开始仔细阅读日记。
日记属于豪宅的最后一位主人，记录了他对这座豪宅和家族历史的研究。
随着你的阅读，你发现一段令人不安的记录：主人提到了一个古老的仪式，据说可以召唤出一个强大的存在来保护家族。但仪式需要一个牺牲。

你感到一阵寒意，但你知道这个信息对你的调查至关重要。

skill: library_use (使用角色卡的技能)
difficulty: medium

请进行技能检定，以确定你是否能从日记中发现更多关于仪式的细节和可能的线索。

I am making a skill check using LIBRARY_USE against a difficulty of MEDIUM.
And I rolled a 87 for a result of FAILURE. TURN: 4 


尽管你努力地试图从这些记录中挖掘更多信息，但你发现自己难以理解其中复杂的细节和含义。日记中的文字似乎围绕着仪式的某些方面故意含糊其辞，或许是出于保护知识不被轻易发现的目的。你的注意力开始分散，感觉时间就这样悄然溜走。

失败解读日记中的更多细节后，你意识到可能需要寻找其他线索或证据来帮助你理解这个仪式和它对豪宅诅咒的可能联系。

你要做什么？

possible actions:
1. 再次尝试理解日记中的内容。（可能需要更高的专注度）
2. 检查书架上是否有隐藏的通道或机关。
3. 离开图书室，探索其他地方。

I select option 2. TURN: 5 


你决定暂时放下日记，转而检查图书室内的书架，希望找到隐藏的通道或机关。
你仔细观察每一排书架，用手轻轻推动一些看起来比较古老的书籍。
经过一番搜索，你的手触碰到了一个书脊，感觉它与众不同。你轻轻地拉动，听到了一声微小的“咔嚓”声。

突然，书架缓缓移动，露出了一个隐藏的通道。通道昏暗而狭窄，但你能看到通道的尽头似乎有光亮透出。

skill: stealth
difficulty: easy

请进行技能检定，以确定你是否能够悄无声息地探索这个隐藏通道，避免潜在的危险或陷阱。

I am making a skill check using STEALTH against a difficulty of EASY.
And I rolled a 75 for a result of SUCCESS. TURN: 6 


你小心翼翼地进入了隐藏通道，确保自己的每一步都尽可能地轻和无声。你的谨慎让你成功地避开了可能存在的陷阱和威胁，没有引起任何不必要的注意。

通道尽头是一扇小门，你轻轻推开，发现自己来到了豪宅的一个未知区域。这里是一个看起来像是实验室的房间，到处都是古老的书籍、奇怪的器具，以及一张中央的工作台。工作台上摆放着各种化学试剂和一本开着的笔记本。

这个房间显然是用来进行某种形式的研究或实验的。你注意到工作台旁边有一个小型的保险箱，看起来很有年头了。

你要做什么？

possible actions:
1. 检查工作台上的笔记本，看看是否有关于仪式或诅咒的信息。
2. 尝试打开保险箱，看看里面隐藏着什么。
3. 离开这个房间，继续探索豪宅的其他部分。

I select option 2. TURN: 7 


你走向保险箱，仔细观察它的外观和锁具。保险箱看起来非常古老，锁上有一个转盘式的组合锁。虽然没有明显的线索表明开锁组合，但你决定尝试一下自己的运气和直觉。

skill: luck
difficulty: hard

请进行技能检定，以确定你是否能够成功地解开保险箱的锁，发现里面可能隐藏的秘密。

I am making a skill check using LUCK against a difficulty of HARD.
And I rolled a 25 for a result of SUCCESS. TURN: 8 


你深呼吸，集中精神尝试着转动保险箱的锁。你的直觉引导着你，手指轻巧地调整着组合锁。几次尝试之后，你听到了那令人兴奋的“咔嚓”声——保险箱被成功解锁！

打开保险箱，你发现里面放着一些文件和一件奇怪的小雕像。文件看起来像是一些日记和信件，详细记录了豪宅主人与一个神秘组织的联系，以及他们试图通过仪式召唤某种力量的计划。小雕像似乎与仪式有关，散发着一种不寻常的氛围。

这些发现让你更加确信，豪宅的诅咒与这个仪式密切相关。你意识到，要解开豪宅的秘密，可能需要更深入地了解这个仪式和神秘组织。

round summary: 在豪宅的图书室找到了一个隐藏通道，通道尽头的房间里，你成功地解开了一个古老保险箱的锁，发现了一些关键文件和一个与仪式有关的小雕像。这些发现为你提供了重要线索，指向了豪宅诅咒背后的真相。

HBox()